# PHYS20762 Computational Physics
## Project 3: Monte Carlo Techniques
### Neutron Transport and Scattering Through a Shielding Layer

Author: David Phelan  
The University of Manchester  
Date: May 2025  

---

## 1. Introduction

In this project, we use Monte Carlo techniques to simulate the behaviour of thermal neutrons passing through various shielding materials. The aim is to model neutron scattering, absorption, and transmission processes within slabs of water, lead, and graphite.

By generating random numbers, random points in three dimensions, and implementing exponential distributions, we build up to a full neutron transport simulation. We will visualise random walks, quantify transmission, reflection, and absorption rates, and determine the characteristic attenuation lengths for different materials.

### 1.1 Objectives

  1. Verify the generation of uniform random numbers using `numpy.random.uniform()`.
  2. Generate and visualise random 3D points to assess distribution properties.
  3. Create and test an exponential random number generator.
  4. Produce isotropic random directions and isotropic steps with exponentially distributed lengths.
  5. Simulate neutron random walks through slabs of different materials.
  6. Quantify neutron absorption, reflection, and transmission rates as functions of slab thickness.
  7. Determine attenuation lengths from transmission data.
  8. Implement and verify the Woodcock method for simulating neutron transport through two adjacent materials.

Throughout the project, we aim to support the simulation with appropriate plots and discuss the numerical results, including any uncertainties.

## 2. Setup & Initialisation

1. Using pip to install essential libraries
1. Importing the required libraries, functions, and types

In [ ]:
%pip install numpy scipy plotly nbformat pandas

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from collections import Counter
from IPython.display import display, Markdown
import textwrap
import time
import math

### Random Seed for Reproducibility
To ensure consistent and reproducible results across different runs of the notebook, we set a fixed random seed at the start of the script. This guarantees that each random number generated will follow the same sequence every time the notebook is executed (assuming each cell is executed once in order).

In [ ]:
np.random.seed(42)

To track the performance of the simulation, we use Python’s `time` module to measure runtime by recording the start time and immediately printing the elapsed time after execution.

In [ ]:
start = time.time()

### Helpers

`print_markdown` is a small helper function to print markdown. It's used for displaying results at various points during the project.

`textwrap.dedent` is used to support the case where you use triple-quoted strings over multiple lines indented, for example:

```python
def print_list():
    print_markdown(f"""
    - Point 1
    - Point 2
    """)
```

would output:

```markdown
- Point 1
- Point 2
```

without `textwrap.dedent`, and by including it we get the expected output:

- Point 1
- Point 2

In [ ]:
def print_markdown(markdown):
    display(Markdown(textwrap.dedent(markdown)))

`plot` provides a consistent rendering experience that can be adjusted to suit the user's display. If the figures are too large or small in one dimension for your monitor, please adjust the values here.

In [ ]:
def plot(figure, figures=1):
    figure.update_layout(
        autosize=True,
        width=(1920/2) * figures,
        height=1080/2,
    )
    figure.show()
    return figure

`format_appropriate` is used to format numbers to an appropriate precision, given their error:

In [ ]:

def format_appropriate(value, error, sig=2, latex=True):
    def round_sig(x, sig=2):
        if x == 0:
            return 0.0
        else:
            return round(x, sig - int(math.floor(math.log10(abs(x)))) - 1)

    def format_sig(x, sig=2):
        if x == 0:
            return f"{0:.{sig - 1}f}"
        else:
            digits = sig - int(math.floor(math.log10(abs(x)))) - 1
            return f"{x:.{max(digits, 0)}f}"

    rounded_error = round_sig(error, sig)
    if rounded_error == 0:
        return f"{value}", f"{rounded_error}"

    decimal_places = -int(math.floor(math.log10(abs(rounded_error)))) + (sig - 1)
    rounded_value = round(value, decimal_places)
    if latex:
        return f"${rounded_value:.{decimal_places}f} \\pm {format_sig(rounded_error, sig)}$"
    return f"{rounded_value:.{decimal_places}f} ± {format_sig(rounded_error, sig)}"



## 3. Simulation Classes

To manage the material properties and slab geometries systematically, we define two classes: `Material` and `Slab`.

### `Material` Class

The `Material` class encapsulates the physical properties needed to simulate neutron interactions, including:

- **Scattering cross-section** ($σ_s$)
- **Absorption cross-section** ($σ_a$)
- **Total cross-section** (sum of scattering and absorption)
- **Absorption probability**

It also includes a `construct_from_microscopic_properties` method to conveniently calculate macroscopic cross-sections based on:

- `density` $\rho$
- `molar_mass` $M$
- Microscopic cross-sections (given in barns, converted to ${cm}^2$)

This design improves clarity and avoids recalculating these properties throughout the notebook.

In [ ]:
class Material:
    BARN_TO_CM2 = 1e-24  # cm²
    AVOGADRO_CONSTANT = 6.02214076e23  # 1/mol

    def __init__(self, name, scattering_cross_section, absorption_cross_section):
        self.name = name
        self.scattering_cross_section = scattering_cross_section
        self.absorption_cross_section = absorption_cross_section
        self.total_cross_section = scattering_cross_section + absorption_cross_section
        self.absorption_probability = absorption_cross_section / self.total_cross_section if self.total_cross_section > 0 else 0

    @staticmethod
    def construct_from_microscopic_properties(name, sigma_a, sigma_s, density, molar_mass):
        number_density = Material.AVOGADRO_CONSTANT * density / molar_mass
        

        return Material(
            name,
            number_density * sigma_s * Material.BARN_TO_CM2,
            number_density * sigma_a * Material.BARN_TO_CM2
        )

### `Slab` Class

The `Slab` class represents a slab of shielding material, characterised by:

- A `Material` object (e.g., water, lead, graphite)
- A `width` (thickness $L$) through which neutrons travel

By creating a separate `Slab` class, we make the simulation modular, allowing us to easily vary material type and slab thickness during experiments.

In [ ]:
class Slab:
    def __init__(self, material, width):
        self.material = material
        self.width = width

### Initialisation of Materials

We instantiate the following materials using their known physical properties:

In [ ]:
water = Material.construct_from_microscopic_properties(
    "Water", 0.6652, 103.0, 1.0, 18.0153)

lead = Material.construct_from_microscopic_properties(
    "Lead", 0.158, 11.221, 11.35, 207.2)

graphite = Material.construct_from_microscopic_properties(
    "Graphite", 0.0045, 4.74, 1.67, 12.011)

materials = [water, lead, graphite]

Each material's macroscopic properties are now readily accessible for subsequent neutron transport simulations.

---

## 4. Testing Random Number Generators

### 4.1 Uniform Random Number Generator (`numpy.random.uniform`)

We first test Python's built-in uniform random number generator, `numpy.random.uniform()`. 
In one dimension, a uniform random generator should produce a flat probability density between 0 and 1. 
In three dimensions, random (x, y, z) points should fill the space evenly without any clustering, streaks, or artefacts. 
We visualise the results using a histogram of random values and a 3D scatter plot.

In [ ]:
def test_random_number_generator():
    x = np.random.uniform(0, 1, 1000)
    y = np.random.uniform(0, 1, 1000)
    z = np.random.uniform(0, 1, 1000)

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Uniform)", "3D Scatter (Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Uniform 3D Points"
        ),
        row=1, col=2
    )

    plot(fig, figures=2)

test_random_number_generator()

### 4.2 Biased Random Number Generator (`RandSSP`)

Compare the results above to the same plots using a flawed random number generator, `RandSSP`, which is known to introduce bias into random samples.
Such flaws can lead to visible structure in the generated points, such as banding or clustering, which should not occur for truly random distributions.

In [ ]:
class RandSSP:
    def __init__(self, seed=123456789):
        self.m = 2**31
        self.a = 2**16 + 3
        self.c = 0
        self.x = seed

    def generate(self, p, q):
        r = np.zeros((p, q))
        for l in range(q):
            for k in range(p):
                self.x = (self.a * self.x + self.c) % self.m
                r[k, l] = self.x / self.m
        return r

In [ ]:
def test_random_number_generator():
    r = RandSSP().generate(3, 1500)
    x, y, z = r[0, :], r[1, :], r[2, :]

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'xy'}, {'type': 'scene'}]],
        subplot_titles=("Histogram (Non-Uniform)", "3D Scatter (Non-Uniform)")
    )

    fig.add_trace(
        go.Histogram(x=x, nbinsx=20, name="Non-Uniform Histogram"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=2),
            name="Non-Uniform 3D Points"
        ),
        row=1, col=2
    )

    plot(fig, figures=2)

test_random_number_generator()

The `RandSSP` generator produces clear evidence of bias. With the 2D histogram it is difficult to see the bias, but plotting the values in 3 dimensions reveals points aligned in planes, highlighting structural artefacts.

## 5 Penetration Depths (Excluding Scattering)

We begin by simulating neutron penetration depths into various materials, assuming **pure absorption** with **no scattering**. In this simplified model, each neutron travels in a straight line and is absorbed at a random depth determined by the material's **macroscopic absorption cross section**, $\Sigma_a$.

The probability distribution for the depth $x$ at which a neutron is absorbed follows an exponential decay:

$$P(x) = \Sigma_a e^{-\Sigma_a x}$$

This means the number of neutrons surviving at least a distance $x$ (the **survival function**) decreases exponentially.

To sample from this distribution, we generate a uniform random number $r \in (0, 1)$ and apply the transformation:

$$x = \frac{-\ln r}{\Sigma_a}$$

This yields an exponentially distributed set of penetration depths.

#### Histogram-Based Simulation

We repeated this process for 10,000 neutrons per trial and construct a histogram of absorption depths. To reduce statistical noise, we run 10 independent simulations and compute the **mean and standard deviation** of the binned survival function across those trials.

These uncertainties are shown as error bars in the plots and represent the empirical standard deviation of each bin's neutron count, normalised by bin width to yield the probability density function. This method, while more computationally expensive than analytic formulas, provides a reproducible and robust estimate of variability without relying on distributional assumptions.

An alternative method — which is commonly used in similar contexts — would be to apply binomial or Gaussian error propagation formulas based on the number of simulated neutrons. This would significantly reduce computation time but assumes ideal statistical behaviour, which may not always capture the nuances of stochastic simulations involving random walks and boundary conditions.

- **Binomial error (for fate proportions like transmission rate):**
  $$
  \sigma_p = \sqrt{ \frac{p(1 - p)}{n} }
  $$
  where $p$ is the observed probability (e.g. fraction transmitted) and $n$ is the number of simulated neutrons.

- **Gaussian error (for continuous counts or histogram bins):**
  $$
  \sigma = \sqrt{N}
  $$
  where $N$ is the number of counts in the bin.

These formulas provide quick, first-principles error estimates and are valid when the underlying assumptions hold. However, in simulations involving random walks with termination conditions and geometry-dependent behaviour, empirically measured uncertainties may offer a more faithful representation of variability.

An example implementation of the binomial error is shown below.

```python
def binomial_standard_error(probability, trials):
    return np.sqrt(probability * (1 - probability) / trials)
```

### 5.1 Computing Simulations

`transform_and_fit` estimates the attenuation length $\lambda$ by taking the natural logarithm of the mean neutron counts per bin, linearising the function into the form $y = mx + c$:

$$\ln N(x) = -\frac{x}{\lambda} + \ln N_0$$

where:

- $y = \ln N(x)$
- $x$ is the depth
- $m = -1 / \lambda$
- $c = \ln N_0$

We fit this line using weighted least squares, with weights given by the inverse of the relative uncertainty in each bin. The standard deviation of each bin (divided by the mean count) gives us the uncertainty in $\ln N(x)$ via error propagation:

$$\sigma_{\ln N} = \frac{\sigma_N}{N}$$

The slope and its uncertainty are then used to estimate the attenuation length and its error:

$$\lambda = -\frac{1}{m}, \quad \sigma_\lambda = \frac{\sigma_m}{m^2}$$

In [ ]:
def transform_and_fit(depths, mean_counts, std_counts):
    mask = (mean_counts > 0) & (std_counts > 0)
    x = depths[mask]
    y = np.log(mean_counts[mask])
    y_err = std_counts[mask] / mean_counts[mask]

    # Filter out any Inf or NaN values
    finite_mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(y_err) & (y_err > 0)
    x = x[finite_mask]
    y = y[finite_mask]
    y_err = y_err[finite_mask]

    if len(x) < 2:
        raise ValueError("Not enough valid data points to perform fit.")

    weights = 1 / y_err
    fit_coeffs, cov_matrix = np.polyfit(x, y, 1, w=weights, cov=True)

    slope, intercept = fit_coeffs
    slope_err = np.sqrt(cov_matrix[0, 0])
    intercept_err = np.sqrt(cov_matrix[1, 1])

    return x, y, y_err, slope, intercept, slope_err, intercept_err

`compute_histograms` runs a repeated simulation of neutron absorption events for a given material, using the exponential sampling method introduced earlier, generating a histogram of **penetration depths** across multiple trials.

For each trial:
- A number of neutrons (`samples`) are simulated
- The distances are binned into a histogram representing **counts per bin**

Across all `trials`, we:
- Average the bin counts to get the **mean number of absorptions per bin**
- Compute the **standard deviation** across trials in each bin to estimate uncertainty
- Convert both the mean and standard deviation into **probability densities** by dividing by the bin width

The result represents the **probability per unit length** that a neutron is absorbed at a given depth — i.e., the probability density function of absorption. This can be compared directly to the theoretical exponential decay curve for validation.

The function returns:
- The bin midpoints
- The mean and standard deviation of raw bin counts
- The mean and standard deviation of the probability density

In [ ]:
def compute_histograms(samples, material, bins, trials):
    histograms = []
    bin_edges = None
    all_distances = []

    for _ in range(trials):
        distances = -np.log(np.random.uniform(0, 1, samples)) / material.absorption_cross_section
        all_distances.append(distances)

    global_max = 5 / material.absorption_cross_section

    for distances in all_distances:
        counts, bin_edges = np.histogram(distances, bins=bins, range=(0, global_max))
        histograms.append(counts)

    bin_width = bin_edges[1] - bin_edges[0]
    histograms = np.array(histograms)
    mean_counts = np.mean(histograms, axis=0)
    mean_density = mean_counts / bin_width
    std_counts = np.std(histograms, axis=0)
    std_density = std_counts / bin_width

    bin_midpoints = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    return bin_midpoints, mean_counts, std_counts, mean_density, std_density

`plot_mean_histogram` is a simple function for plotting the results obtained as a histogram.

In [ ]:
def plot_mean_histogram(bin_midpoints, mean_density, std_density, material):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x=bin_midpoints,
        y=mean_density,
        error_y=dict(type='data', array=std_density, visible=True),
        name=f"Mean histogram ({material.name})",
        marker=dict(color='rgba(100, 149, 237, 0.7)', line=dict(color='black', width=1))
    ))

    fig.update_layout(
        title=f"Mean Neutron Penetration Histogram for {material.name}",
        xaxis_title="Distance (cm)",
        yaxis_title="Neutron number density (counts per cm)",
        bargap=0.05,
        legend=dict(x=0.65, y=0.98),
        showlegend=True
    )

    plot(fig)

`plot_exponential_fit_with_errors` is a simple function for plotting the transformed data as a straight line fit

In [ ]:
def plot_exponential_fit_with_errors(x, y, y_err, fit_coeff, fit_intercept, material):
    fig = go.Figure()

    # Scatter plot of log(counts) with error bars
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        error_y=dict(type='data', array=y_err, visible=True),
        mode='markers',
        name=f"Log binned counts ({material.name})"
    ))

    # Fitted line
    fitted_y = fit_coeff * x + fit_intercept
    fig.add_trace(go.Scatter(
        x=x,
        y=fitted_y,
        mode='lines',
        name=f"Fit: y = {fit_coeff:.3f}x + {fit_intercept:.3f}"
    ))

    # Horizontal line at log(N0/e) = intercept - 1
    n0_over_e_line = fit_intercept - 1
    fig.add_trace(go.Scatter(
        x=[0, max(x)],
        y=[n0_over_e_line] * 2,
        mode='lines',
        line=dict(color='red', dash='dash'),
        name="log(N₀/e) line"
    ))

    fig.update_layout(
        title=f"Exponential Decay Fit for {material.name}",
        xaxis_title="Distance (cm)",
        yaxis_title="log(Number of neutrons)",
        legend=dict(x=0.65, y=0.98)
    )

    plot(fig)

`print_results` is the final function for this section. It executes the other functions as required graphing the histogram and the straight line fit for water, but skipping it for the other materials (they look similar so to reduce clutter they are ommitted). It then produces a table of results for the attenuation lengths for all materials, although these are far larger than expected since we do not consider scattering as mentioned above, hence the neutrons penetrate much further when only travelling in straight lines.

In [ ]:
def print_results(samples, bins, trials):
    # Water first (for plotting)
    bin_midpoints, mean_counts, std_counts, mean_density, std_density = compute_histograms(samples=samples, material=water, bins=bins, trials=trials)
    plot_mean_histogram(bin_midpoints, mean_density, std_density, material=water)
    x, y, y_err, slope, intercept, slope_err, intercept_err = transform_and_fit(bin_midpoints, mean_counts, std_counts)
    plot_exponential_fit_with_errors(x, y, y_err, slope, intercept, material=water)

    # Get results for all materials
    materials = [water, lead, graphite]
    data = []
    for material in materials:
        bin_midpoints, mean_counts, std_counts, mean_density, std_density = compute_histograms(samples=samples, material=material, bins=bins, trials=trials)
        _, _, _, slope, _, slope_err, _ = transform_and_fit(bin_midpoints, mean_counts, std_counts)
        data.append((material, slope, slope_err))

    # Markdown output
    markdown = "Material | Theoretical Attenuation Length (cm) | Simulated Attenuation Length (cm)\n"
    markdown += " --- | --- | --- \n"
    for material, slope, slope_err in data:
        theoretical = 1 / material.absorption_cross_section
        simulated = -1 / slope
        simulated_err = slope_err / (slope ** 2)
        result = format_appropriate(simulated, simulated_err, 2)
        markdown += f"{material.name} | ${theoretical:.2f}$ | {result}\n"

    print_markdown(markdown)

print_results(samples=10000, bins=50, trials=10)

From the table above, the results agree with the theoretical values, within their statistical errors.

---

## 6. Generating Isotropic Directions

### 6.1 Generating Isotropic Unit Vectors

To simulate random neutron directions uniformly distributed over a sphere, we generate isotropic unit vectors $\vec{r} = x\hat{i} + y\hat{j} + z\hat{k}$ satisfying $|\vec{r}| = 1$. 
We use spherical polar coordinates $(\theta, \phi)$ where:
- $\theta$ is the polar angle, drawn from $\cos\theta \sim \text{uniform}(-1, 1)$,
- $\phi$ is the azimuthal angle, drawn uniformly from $[0, 2\pi)$.

This ensures points are uniformly distributed over the sphere's surface, representing isotropic scattering accurately.

If we sampled $\theta$ uniformly from $[0, \pi]$, the resulting points would cluster near the poles due to the non-uniform area element in spherical coordinates. By sampling $\cos\theta$ uniformly instead, we correct for this bias and ensure an even distribution over the sphere’s surface.

The resulting direction vector $\vec{r}$ is obtained by converting the sampled angles to Cartesian coordinates:

$$
x = \sin\theta \cos\phi, \quad y = \sin\theta \sin\phi, \quad z = \cos\theta
$$

In [ ]:
def generate_random_unit_vector():

    theta = np.arccos(np.random.uniform(-1, 1))
    phi = np.random.uniform(0, 2*np.pi)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    return x, y, z

#### 6.1.1 Visualising the Isotropic Unit Vectors

To verify the uniformity of the generated directions, we visualise many unit vectors in 3D space. 
A truly isotropic distribution should appear as a uniform scatter of points across the surface of a sphere.

In [ ]:
def visualize_random_unit_vectors():
    x,y,z = zip(*[(x,y,z) for x,y,z in [generate_random_unit_vector() for _ in range(10000)]])

    fig = px.scatter_3d(
        x=x, y=y, z=z, color=z,
        labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
    )
    fig.update_traces(marker=dict(size=2))  # Smaller points
    fig.update_layout(
        title="3D Scatter Plot"
    )
    plot(fig)

visualize_random_unit_vectors()

### 6.2 Simulating Random Walks in 3D Space

Using the isotropic unit vectors generated above, we now simulate random neutron paths through 3D space. 
At each step, a neutron travels in a random direction, with a step length sampled from an exponential distribution:

$$P(x) = \frac{1}{\lambda} e^{-x/\lambda}$$

where $\lambda$ is the mean free path determined by the material's macroscopic cross-section.

After each step, the neutron scatters and is redirected in a new random direction, sampled isotropically as described in Section 6.1. This process repeats, simulating the stochastic nature of neutron motion through a scattering medium.

Note: at this stage we are ignoring absorption, and just focusing on the scattering of the neutrons for a set number of paths.

In [ ]:
def random_uniform_path_length(cross_section):
    return -np.log(np.random.uniform(0, 1)) / cross_section

In [ ]:
def random_walk(cross_section):
    x, y, z = (0, 0, 0)
    yield (x, y, z)
    while True:
        step_size = random_uniform_path_length(cross_section)
        dx, dy, dz = generate_random_unit_vector()
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        yield (x, y, z)


#### 6.2.1 Visualising Random Walks

We now simulate neutron random walks through different materials. 
Each trajectory represents the stochastic motion of a neutron, with direction sampled isotropically and step lengths sampled from the material's exponential attenuation law.

In [ ]:
def visualise_random_walk(material):
    macroscopic_cross_section= material.scattering_cross_section
    
    positions_iterator = random_walk(macroscopic_cross_section)
    positions = [next(positions_iterator) for _ in range(100)]

    x, y, z = zip(*positions)
    scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', marker=dict(size=2))
    layout = go.Layout(
        title=f"Random Walk in {material.name}",
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig = go.Figure(data=[scatter], layout=layout)
    plot(fig)

visualise_random_walk(water)

In water, neutrons travel relatively short distances before scattering. This is because water has a high macroscopic scattering cross section, meaning interactions are frequent and the mean free path is short. As a result, the random walk appears tightly confined.

Water serves as a good benchmark for comparison with lead and graphite, which allow neutrons to travel much farther.

In [ ]:
visualise_random_walk(lead)

In lead, neutrons travel significantly farther than in water — roughly 10 times the distance on average.

In [ ]:
visualise_random_walk(graphite)

Neutrons travel even farther in graphite — approximately 2.5 times farther than in lead. This results in extremely long, sparse random walks.

### Summary

The extent of each neutron’s random walk depends directly on the mean free path, which is inversely related to the material’s macroscopic scattering cross section. Water has the shortest mean free path, while graphite has the longest in this set — leading to progressively more extended trajectories from water to lead to graphite.

This ordering aligns with theoretical expectations. The mean free path is inversely proportional to the likelihood of scattering, governed by the material’s macroscopic scattering cross section. Water, being hydrogen-rich, has a high number density of effective scatterers, leading to frequent collisions. Lead, despite its high mass density, has a relatively low number density of scattering centres and a smaller cross section for neutrons, resulting in fewer interactions. Graphite, with its light nuclei and sparse structure, allows neutrons to travel the farthest between collisions.

---

### 6.3 Visualising Neutron Random Walks in a Slab

We now simulate neutron random walks through a finite slab of material. 
Each slab is placed between two planes located at $x = 0$ and $x = L$, where $L$ is the slab thickness. 

Initially, neutrons are fired **perpendicularly into the slab** from the left boundary at $x = 0$, moving along the positive $x$-axis. 
Each neutron then undergoes a stochastic random walk within the slab:
- The step size is sampled from the same exponential distribution used in Section 6.2, with mean free path $\lambda$ determined by the material's macroscopic cross-section.
- After each step, the neutron's new position is updated by moving along a randomly generated isotropic unit vector.

At each step, we determine whether the neutron:
- **Is absorbed** within the slab (random survival check against absorption probability),
- **Scatters** and continues walking within the slab,
- **Escapes** by crossing beyond the slab boundaries (either $x < 0$ or $x > L$).

The random walk terminates if the neutron is absorbed or escapes.
We visualise individual neutron trajectories through the slab for different materials, illustrating the stochastic scattering and absorption behaviour in each case.

The simple `random_walk` function is now replaced with a version that takes a slab of material and performs a full walk until an end condition (escape or absorb) is met.

In [ ]:
def random_walk(slab):
    result = []
    x, y, z = (0, 0, 0)
    result.append((x, y, z))
    dx, dy, dz = (1, 0, 0)  # Initial direction
    while True:
        step_size = random_uniform_path_length(slab.material.total_cross_section)
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        if x > slab.width:
            return "Transmitted", result
        elif x < 0:
            return "Reflected", result
        elif np.random.uniform(0, 1) < slab.material.absorption_probability:
            return "Absorbed", result
        else:
            # Scattering occurs
            dx, dy, dz = generate_random_unit_vector()
        result.append((x, y, z))

`visualise_random_walks` plots the requested number of random walks through a given slab of material, colour coding them with the fate of the walk (Transmitted, Reflected, or Absorbed).

Note: each walk is written on the plot as a separate 3D scatter so some adjustments are made to reduce clutter in the legend to only show a record for the first walk of each fate.

In [ ]:
def visualise_random_walks(slab, number_of_neutrons):
    walks = [ random_walk(slab) for _ in range(number_of_neutrons) ]

    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )

    show_legend = {
        "Transmitted": True,
        "Reflected": True,
        "Absorbed": True
    }
    colours = {
        "Transmitted": "blue",
        "Reflected": "red",
        "Absorbed": "green"
    }
    fate_counts = Counter([walk[0] for walk in walks])

    def walk_to_scatter(walk):
        x, y, z = zip(*walk[1])
        showlegend = show_legend[walk[0]]
        show_legend[walk[0]] = False # only show the legend for the first walk of each type
        return go.Scatter3d(
            x=x,
            y=y,
            z=z,
            mode='markers+lines',
            marker=dict(size=2),
            name=f'{walk[0]} ({fate_counts[walk[0]]})',
            line=dict(color=colours[walk[0]]),
            showlegend=showlegend)
    
    fig = go.Figure(data=[walk_to_scatter(walk) for walk in walks], layout=layout)
    fig.update_layout(
        title=f"Random Walk in {slab.material.name}"
    )
    plot(fig)

visualise_random_walks(Slab(water, 4), 30)


### 6.4 Quantifying Neutron Scattering, Absorption, and Transmission

We now quantify the outcomes of neutron interactions with the slab by simulating large numbers of random walks are recording how many neutrons are:
- **Absorbed** within the slab,
- **Reflected** back through the entrance at $x < 0$,
- **Transmitted** through the far side at $x > L$.

We record the number of neutrons absorbed ($N_A$), reflected ($N_R$), and transmitted ($N_T$), and compute the corresponding rates:

$$\text{Absorption Rate} = \frac{N_A}{N}, \quad \text{Reflection Rate} = \frac{N_R}{N}, \quad \text{Transmission Rate} = \frac{N_T}{N}$$

To ensure our results are statistically robust, we repeat the simulation multiple times and compute the **mean and standard deviation** of the transmission, reflection, and absorption probabilities.

`random_walks` performs multiple neutron random walk simulations through a specified slab of material. It returns a list of individual walk outcomes by repeatedly calling the `random_walk` function the requested number of times.

In [ ]:
def random_walks(slab, counts):
    return [random_walk(slab) for _ in range(counts)]

`count_fates` simulates a given number of random walks through a slab and returns a count of how many neutrons were transmitted, reflected, or absorbed. It uses the `Counter` class to tally the first element of each walk result, which encodes the neutron's final fate.

In [ ]:
def count_fates(slab, counts):
    return Counter([walk[0] for walk in random_walks(slab, counts)])

`aggregate_trials` processes the results of multiple random walk trials and computes the mean and standard deviation of each fate type (Transmitted, Reflected, or Absorbed). It returns the estimated probability and associated uncertainty for each fate by averaging across trials and normalising by the number of neutrons simulated per trial.

In [ ]:
def aggregate_trials(trial_results, number_of_neutrons):
    fates = { }
    for run in trial_results: 
        for fate in run:
            if fate not in fates:
                fates[fate] = []
            fates[fate].append(run[fate])
    for fate in fates:
        probability = np.mean(fates[fate]) / number_of_neutrons
        error = np.std(fates[fate]) / number_of_neutrons
        yield fate, probability, error

`quantify_random_walk` estimates the probabilities of transmission, reflection, and absorption by performing multiple trials of random walk simulations through a given slab. Each trial simulates a fixed number of neutrons, and the aggregated results are processed to return the mean probability and uncertainty for each fate.

In [ ]:
def quantify_random_walk(slab, number_of_neutrons, trials):
    runs = [count_fates(slab, number_of_neutrons) for _ in range(trials)]
    return aggregate_trials(runs, number_of_neutrons)

`show_quantified_random_walk` visualises the outcomes of random walk simulations for one or more slab configurations. For each slab, it estimates the transmission, reflection, and absorption rates using multiple trials and displays the results as a grouped bar chart, with error bars representing the uncertainty in each fate percentage. We then call this function passing 10cm of each of the three materials.

In [ ]:

def show_quantified_random_walk(slabs, number_of_neutrons, trials):
    fig = go.Figure()
    for slab in slabs:
        fates = quantify_random_walk(slab, number_of_neutrons, trials)
        fates = list(fates)
        fates = sorted(fates, key=lambda x: x[0])
        fates = [(fate[0], fate[1] * 100, fate[2] * 100) for fate in fates]
        fates = [(fate[0], fate[1], fate[2]) for fate in fates if fate[1] > 0.01]

        fig.add_trace(go.Bar(
            x=[fate[0] for fate in fates],
            y=[fate[1] for fate in fates],
            name=f"{slab.material.name} ({slab.width} cm)",
            error_y=dict(type='data', array=[fate[2] for fate in fates])
        ))

    fig.update_layout(
        title="Quantified Random Walk Results",
        xaxis_title="Fate",
        yaxis_title="Rate (%)",
        barmode='group',
        legend=dict(x=0.68, y=0.98),
        yaxis=dict(range=[0, 100])
    )
    plot(fig)

show_quantified_random_walk([Slab(water, 10), Slab(graphite, 10), Slab(lead, 10)], number_of_neutrons=1000, trials=10)

The results for 1000 neutrons are shown above. The majority of the neutrons are reflected in all materials, which is expected given the relatively large thickness of the slabs (10 cm).

Graphite in particular shows how the relatively low absorption cross section and long mean free path lengths translate into very few neutrons being absorbed.

### 6.5 Variation of Neutron Interaction Rates with Slab Thickness

To analyse how neutron behaviour varies with slab thickness $L$, we perform repeated random walk simulations for a range of thicknesses. For each slab thickness, the absorption, reflection, and transmission rates are computed as before.

#### Error-Based Convergence

Rather than fixing the number of simulation batches in advance, we continue running batches of simulations **until the standard deviation of each outcome falls below a target threshold** (we have chosen 0.03 as a good tradeoff between accuracy and computation time). This ensures we adaptively allocate more computation only where necessary, such as around the critical transition regions where transmission probabilities drop off sharply.

This approach allows us to maintain statistical accuracy **without wasting computation** on slabs that already give stable results.

#### Performance Tuning

- The `trials` parameter controls the number of independent simulations per batch (default is 10).
- The `batch_size` parameter controls how often convergence is checked. We typically use a small value (e.g. 5), since the **random walk simulation dominates runtime**, and large batches could overshoot the desired accuracy. Smaller batches allow earlier convergence detection.
- On slower machines, you can raise the target error (e.g. from 0.03 to 0.05) or reduce the number of steps to improve speed, at the cost of increased uncertainty.

#### Attenuation Length Estimation

To estimate the attenuation length $\lambda$, we focus on the transmission rate as a function of slab thickness. According to theory, the transmission should decay exponentially:

$$T(x) = \exp\left(-\frac{x}{\lambda}\right)$$

We transform this into a linear function:

$$\ln T(x) = -\frac{x}{\lambda} + \ln T_0$$

This allows us to perform a **weighted linear regression** (not a nonlinear curve fit) on the logarithm of the transmission values, using the standard deviation of each point as the fitting weight.

From the fitted slope $m$, we compute the attenuation length:

$$\lambda = -\frac{1}{m}, \quad \sigma_\lambda = \frac{\sigma_m}{m^2}$$

Where $\sigma_m$ is the standard error in the fitted slope. This method ensures a consistent and statistically grounded estimation of attenuation behaviour across all materials.


In [ ]:
def plot_width_vs_fates(trials, steps, materials, target_error, batch_size):
    for material, max_width in materials:
        widths = np.linspace(0, max_width, steps)
        data = {}

        for width in widths:
            fate_counts = [Counter() for _ in range(trials)]
            runs = 0
            converged = False

            while not converged and runs < 10_000:
                for _ in range(batch_size):
                    batch = [random_walk(Slab(material, width)) for _ in range(trials)]
                    for i, (fate_name, _) in enumerate(batch):
                        fate_counts[i][fate_name] += 1
                    runs += 1

                all_fates = sorted({f for fc in fate_counts for f in fc})
                converged = True
                for fate in all_fates:
                    proportions = [fc.get(fate, 0) / runs for fc in fate_counts]
                    std = np.std(proportions)
                    if std >= target_error or runs == 1:
                        converged = False
                        break

            for fate in all_fates:
                proportions = [fc.get(fate, 0) / runs for fc in fate_counts]
                mean = np.mean(proportions)
                std = np.std(proportions)

                if fate not in data:
                    data[fate] = []
                data[fate].append((width, mean, std))

        # Create two subplots: one for fate probabilities, one for the exponential fit
        fig = make_subplots(
            rows=1, cols=2,
            specs=[[{'type': 'xy'}, {'type': 'xy'}]],
            subplot_titles=(
                f"Fate Probabilities for {material.name}",
                f"Transmission Fit for {material.name}"
            )
        )

        # Left subplot: Fate probabilities (with error bars)
        for fate_name in sorted(data):
            x, y, error = zip(*data[fate_name])
            fig.add_trace(go.Scatter(
                x=x, y=y, mode='markers',
                name=f"{material.name} {fate_name}",
                error_y=dict(type='data', array=error, visible=True)
            ), row=1, col=1)

        # Right subplot: Logarithmic transmission fit

        # Extract transmission data and fit a line to log-transformed values
        transmitted_widths, transmitted_counts, transmitted_errors = zip(*data["Transmitted"])
        x = np.array(transmitted_widths)
        y = np.array(transmitted_counts)
        y_err = np.array(transmitted_errors)
        log_y = np.log(y)
        log_y_err = y_err / y

        # Fit line: log(T) = -x / λ + log(T₀)
        slope, intercept, slope_err, intercept_err = transform_and_fit(x, y, y_err)[3:]

        # Add log-transformed data with error bars
        fig.add_trace(go.Scatter(
            x=x,
            y=log_y,
            mode='markers',
            name="Log Transmission",
            error_y=dict(type='data', array=log_y_err, visible=True)
        ), row=1, col=2)

        # Add fitted line to right subplot
        fitted_log_y = slope * x + intercept
        fig.add_trace(go.Scatter(
            x=x,
            y=fitted_log_y,
            mode='lines',
            name="Linear fitted transmission probability"
        ), row=1, col=2)

        # Compute and overlay theoretical curve on left subplot
        attenuation_length = -1 / slope
        attenuation_length_error = slope_err / (slope ** 2)

        theoretical_x = np.linspace(0, max_width, 1000)
        theoretical_y = np.exp(-theoretical_x / attenuation_length)
        fig.add_trace(go.Scatter(
            x=theoretical_x,
            y=theoretical_y,
            mode='lines',
            name=f"Exponential fitted transmission probability"
        ), row=1, col=1)

        # Vertical line at attenuation length (left subplot)
        fig.add_trace(go.Scatter(
            x=[attenuation_length, attenuation_length],
            y=[0, 1],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False
        ), row=1, col=1)

        # Annotation with attenuation length (right subplot)
        fig.add_annotation(
            x=attenuation_length,
            y=0.8,
            text=f"λ = {format_appropriate(attenuation_length, attenuation_length_error, latex=False)}",
            showarrow=True,
            arrowhead=2,
            ax=-110,
            ay=-40,
            row=1, col=1
        )

        # Shared layout
        fig.update_layout(
            width=1100,
            height=500,
            margin=dict(r=100),
            legend=dict(x=1.05, y=1),
            title=f"Neutron Fate Breakdown and Transmission Fit — {material.name}"
        )

        plot(fig, figures=2)

plot_width_vs_fates(trials=10, steps=30, materials=[(water, 1), (lead, 10), (graphite, 10)], target_error=0.03, batch_size=5)

While the theoretical model assumes that neutron transmission through a material decreases exponentially with thickness, our simulation reveals a subtle but consistent deviation from this idealised behaviour. When plotting the number of transmitted particles as a function of material thickness, the observed data does not follow a perfect exponential trend — particularly at larger thicknesses.

This deviation is small but measurable, and suggests that the ideal exponential model does not fully capture the behaviour of our stochastic simulation. The precise cause of this discrepancy is unclear, but it is likely related to the cumulative effects of random walk geometry, absorption probabilities, and simulation boundary conditions.

Importantly, if the exponential fit is restricted to slab thicknesses near or below the attenuation length, the agreement with theory improves significantly. This approach yields a more reliable estimate of the attenuation length and demonstrates the value of localised fitting in complex stochastic systems.


In [ ]:
print_markdown(f"Execution time: ${time.time() - start:.2f}$ s")


---

## 7. Simulating Transmission Through Adjacent Slabs Using the Woodcock Method

In this extension task, we apply the **Woodcock method** (also known as delta-tracking) to simulate neutron transmission through multiple adjacent slabs composed of different materials.

Neutrons are fired perpendicularly into the slabs at $x = 0$ and undergo stochastic random walks based on the Woodcock sampling method.

### Application of the Woodcock Method

First, we compare the **total macroscopic cross-sections** of the materials and identify the maximum value, $\Sigma_{\text{max}}$. This value is used for sampling step lengths throughout the entire simulation, ensuring uniform handling of different materials.

At each step:
- A random step length is drawn from an exponential distribution based on $\Sigma_{\text{max}}$.
- The neutron moves along a randomly generated isotropic direction.

After each step:
  - A uniform random number between 0 and 1 is generated.
  - If the random number is **less than** the **acceptance probability** (defined as $\Sigma_{\text{material}} / \Sigma_{\text{max}}$), the interaction is considered **real**.
    - In this case, we proceed to determine whether the neutron is absorbed, transmitted, or reflected.
  - If the random number is **greater than** the acceptance probability, the interaction is treated as a **virtual collision**.
    - No physical interaction (absorption or scattering) occurs.
    - The neutron simply continues with a new random step.

- If the neutron is in the most dense material in the supplied slabs (i.e., $\Sigma = \Sigma_{\text{max}}$), every interaction is automatically accepted without rejection sampling due to the probability of the collision being real `p_real` having the value 1.

Boundary conditions are handled as follows:
- If the neutron moves beyond $x > 2L$, it is considered **transmitted**.
- If it moves back through $x < 0$, it is considered **reflected**.
- If it undergoes a real collision inside the slab, absorption or scattering is determined based on the material properties.

This approach allows efficient and unbiased simulation of neutron transport through inhomogeneous materials with different cross-sections.

In [ ]:
def random_walk_woodcock(slabs):
    x, y, z = (0, 0, 0)
    dx, dy, dz = (1, 0, 0)
    path = [(x, y, z)]
    max_cross_section = max(slab.material.total_cross_section for slab in slabs)
    total_width = sum(slab.width for slab in slabs)

    def get_slab_map():
        x_start = 0
        for slab in slabs:
            x_end = x_start + slab.width
            yield (x_start, x_end, slab.material, slab.material.total_cross_section / max_cross_section)
            x_start = x_end

    slab_positions = list(get_slab_map())

    def get_material_at(x):
        for start, end, material, p in slab_positions:
            if start <= x < end:
                return (material, p)

    while True:
        step_size = random_uniform_path_length(max_cross_section)
        x_new = x + step_size * dx
        y_new = y + step_size * dy
        z_new = z + step_size * dz
        x, y, z = (x_new, y_new, z_new)

        if x_new < 0:
            path.append((x,y,z))
            return "Reflected", path
        if x_new > total_width:
            path.append((x,y,z))
            return "Transmitted", path

        material, p_real = get_material_at(x_new)
        if material is None:
            raise ValueError("Position out of bounds")

        if np.random.uniform(0, 1) > p_real:
            continue
        else:
            if np.random.uniform(0, 1) < material.absorption_probability:
                path.append((x,y,z))
                return "Absorbed", path
            else:
                dx, dy, dz = generate_random_unit_vector()
                path.append((x,y,z))

In [ ]:
def print_fate_rates(slabs, trials):
    counts = Counter([walk[0] for walk in [random_walk_woodcock(slabs) for _ in range(trials)]])

    markdown = "Fate | Count | Rate (%) \n"
    markdown += "--- | --- | --- \n"
    for fate, count in counts.items():
        rate = count / trials * 100
        markdown += f"{fate} | {count} | {rate:.2f} \n"
    markdown += "\n\n"
    print_markdown(markdown)

print_fate_rates([Slab(graphite, 10), Slab(lead, 10)], 3000)

### 7.1 Validation of the Woodcock Method Implementation

We will now run a series of tests to check that the data produced by the woodcock method produces statistically indistinguishable results when compared to eithe the simple method or an alternative woodcock. These tests taken together should provide confidence the logic is correct.

First we create a helper function `print_fate_rate_comparison` to produce these comparisons, given two tests.

In [ ]:
def print_fate_rate_comparison(walk_func1, walk_func2, number_of_neutrons, trials):
    runs1 = [Counter([walk_func1()[0] for _ in range(number_of_neutrons)]) for _ in range(trials)]
    runs2= [Counter([walk_func2()[0] for _ in range(number_of_neutrons)]) for _ in range(trials)]
    fates1 = list(aggregate_trials(runs1, number_of_neutrons))
    fates2 = list(aggregate_trials(runs2, number_of_neutrons))

    all_fates = sorted(set([fate for fate, _, _ in fates1]) | set([fate for fate, _, _ in fates2]))

    markdown = (
        "| Fate | Trial A Rate (%) ± Error | Trial B Rate (%) ± Error | Δ (%) | Combined Error | Δ / σ |\n"
        "|------|---------------------------|---------------------------|--------|----------------|--------|\n"
    )

    def get_stats(fate, fates):
        for current_fate, probability, error in fates:
            if fate == current_fate:
                return probability, error
        return 0, 0

    for fate in all_fates:
        probability1, error1 = get_stats(fate, fates1)
        probability2, error2 = get_stats(fate, fates2)
        probability1, error1, probability2, error2 = probability1 * 100, error1 * 100, probability2 * 100, error2 * 100

        diff = abs(probability1 - probability2)
        combined_error = np.sqrt(error1**2 + error2**2)
        z_score = diff / combined_error if combined_error != 0 else float('inf')

        markdown += (
            f"{fate} | {format_appropriate(probability1, error1)} | {format_appropriate(probability2, error2)} | "
            f"{diff:.2f} | {combined_error:.2f} | {z_score:.2f} |\n"
        )

    print_markdown(markdown)

#### Split-Slab Equivalence Test

A slab of 5 cm of a material followed by another 5 cm of the same material should be statistically indistinguishable from a single 10 cm slab of that material.

This test compares Woodcock tracking on two 5 cm slabs against simple random walk on one 10 cm slab.

The results below validate that the Woodcock method does not introduce artefacts due to boundary crossings.

In [ ]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(water, 3)]),
    lambda: random_walk(Slab(water, 6)),
    number_of_neutrons=500,
    trials=5
)

#### Split vs Single Slab (Same Method)

This test verifies that two consecutive 3 cm slabs of water produce statistically indistinguishable results from a single 6 cm slab, **using the same Woodcock tracking method** in both cases.

Since the material and total thickness are identical, any differences would indicate that the simulation is sensitive to artificial slab boundaries — which should not be the case.

The results below confirm that the Woodcock method maintains consistency across slab segmentation.


In [ ]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(water, 3)]),
    lambda: random_walk_woodcock([Slab(water, 6)]),
    number_of_neutrons=500,
    trials=5
)

#### Woodcock vs simple (Same Geometry)

This test compares the fate statistics for a 10 cm water slab using both Woodcock and the simple random walk.

Identical geometry and material ensure that any difference arises purely from the tracking method itself. Close agreement validates the correctness of the Woodcock implementation.

In [ ]:
print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 6)]),
    lambda: random_walk(Slab(water, 6)),
    number_of_neutrons=500,
    trials=5
)

#### Vacuum Sandwich Test

This test validates that inserting a vacuum layer between two slabs of identical material does not alter the final outcome.

Physically, a vacuum introduces no scattering or absorption, and should not impact the overall transmission, reflection, or absorption rates.

The test compares 5 cm water + 5 cm vacuum + 5 cm water against a single 10 cm water slab using Woodcock tracking throughout.

In [ ]:
vacuum = Material("Vacuum", 0, 0)

print_fate_rate_comparison(
    lambda: random_walk_woodcock([Slab(water, 3), Slab(vacuum, 5), Slab(water, 3)]),
    lambda: random_walk_woodcock([Slab(water, 6)]),
    number_of_neutrons=500,
    trials=5
)

From the results, not every test was within 1 standard deviation, but we wouldn't expect them to be and increasing the number of trails, although computationally expensive, does show good agreement. These tests give us confidence that the implemented Woodcock method does have many qualities we would expect it to have, and gives consistent results in the cases we can test.

In [ ]:
print_markdown(f"Execution time: ${time.time() - start:.2f}$ s")